<a href="https://colab.research.google.com/github/CharlesR-W/HFCourses/blob/main/gtzan_finetune_HFAHO2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers
!pip install gradio
!pip install evaluate

from huggingface_hub import notebook_login

notebook_login()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [ ]:
from transformers import AutoFeatureExtractor

#model_id = "ntu-spml/distilhubert"
model_id = "mit/ast-finetuned-audioset-10-10-0.4593"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_id,do_normalize=True, return_attention_mask=True)

max_duration = 30.0

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=False,
    )
    return inputs

from datasets import load_dataset

gtzan = load_dataset("marsyas/gtzan", "all")
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
from datasets import Audio

gtzan = gtzan.cast_column("audio", Audio(sampling_rate=feature_extractor.sampling_rate))
gtzan_encoded = gtzan.map(preprocess_function, remove_columns=["audio", "file"], batched=True, batch_size=100, num_proc=1)
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

ind2label_fn = gtzan["train"].features["genre"].int2str
#ind2label_fn(gtzan["train"][0]["genre"])

id2label = {lv: gtzan_encoded["train"].features["label"].int2str(lv) for lv,_ in enumerate(gtzan_encoded["train"].features["label"].names)}
label2id = {v:k for k,v in id2label.items()}

from transformers import AutoModelForAudioClassification
num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_id, num_labels=num_labels, label2id=label2id, id2label=id2label,ignore_mismatched_sizes=True
)



Map:   0%|          | 0/899 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Some weights of ASTForAudioClassification were not initialized from the model checkpoint at mit/ast-finetuned-audioset-10-10-0.4593 and are newly initialized because the shapes did not match:
- classifier.dense.bias: found shape torch.Size([527]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.dense.weight: found shape torch.Size([527, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for name, _ in model.named_parameters():
    print(name)

hubert.masked_spec_embed
hubert.feature_extractor.conv_layers.0.conv.weight
hubert.feature_extractor.conv_layers.0.layer_norm.weight
hubert.feature_extractor.conv_layers.0.layer_norm.bias
hubert.feature_extractor.conv_layers.1.conv.weight
hubert.feature_extractor.conv_layers.2.conv.weight
hubert.feature_extractor.conv_layers.3.conv.weight
hubert.feature_extractor.conv_layers.4.conv.weight
hubert.feature_extractor.conv_layers.5.conv.weight
hubert.feature_extractor.conv_layers.6.conv.weight
hubert.feature_projection.projection.weight
hubert.feature_projection.projection.bias
hubert.encoder.pos_conv_embed.conv.bias
hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0
hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original1
hubert.encoder.layer_norm.weight
hubert.encoder.layer_norm.bias
hubert.encoder.layers.0.attention.k_proj.weight
hubert.encoder.layers.0.attention.k_proj.bias
hubert.encoder.layers.0.attention.v_proj.weight
hubert.encoder.layers.0.attention.v_

In [ ]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size=8
gradient_accumulation_steps=1
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan-run4-ast",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=True,
    report_to=[]
)

import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

from transformers import Trainer
from transformers import default_data_collator

# Initialize our Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    data_collator=default_data_collator,
    tokenizer=None,
    compute_metrics=compute_metrics,
)

<ipython-input-14-4db17ec30f00>:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.384400,0.570946,0.770000
2,0.992700,0.710565,0.740000
3,0.001100,0.541712,0.850000
4,0.000300,0.440193,0.900000
5,0.000900,0.411209,0.890000
6,0.000200,0.709216,0.830000
7,0.000500,0.533362,0.890000
8,0.000100,0.515498,0.890000
9,0.000200,0.508110,0.900000
10,0.000000,0.505026,0.900000


TrainOutput(global_step=1130, training_loss=0.21572997408715375, metrics={'train_runtime': 1550.9167, 'train_samples_per_second': 5.797, 'train_steps_per_second': 0.729, 'total_flos': 6.094112254328832e+17, 'train_loss': 0.21572997408715375, 'epoch': 10.0})

In [ ]:
val_accuracy = trainer.evaluate(gtzan_encoded["test"])["eval_accuracy"]
print(f"Validation accuracy: {val_accuracy}")

kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "model_name": f"{model_name}-finetuned-gtzan",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}

upload_thresh = 0.87
if val_accuracy >= upload_thresh:
    trainer.push_to_hub(**kwargs)
else:
    print(f"Validation accuracy {val_accuracy} is below the upload threshold {upload_thresh}.")



Validation accuracy: 0.9
